In [81]:
from contextlib import contextmanager

@contextmanager
def test():
    try:
        print('connected')
        yield 1
        print('olll')
    finally:
        print('disconnect')


In [82]:
with test() as conn:
    print(conn)

connected
1
olll
disconnect


In [89]:
class CustomContextManager:
    def __init__(self, func):
        self.func = func
        self.gen = None
    
    def __enter__(self):
        print('enter')
        self.gen = self.func()
        return next(self.gen)
    
    def __exit__(self, *args, **kwargs):
        print('exit')
        try:
            next(self.gen)
        except StopIteration:
            pass
        finally:
            self.gen.close()

    def __call__(self):
        return self

@CustomContextManager
def test():
    try:
        print('connected')
        yield 1
        print('olll')
    finally:
        print('disconnect')

with test() as conn:
    print('....')

enter
connected
....
exit
olll
disconnect


In [121]:
import sqlite3
from contextlib import contextmanager

DB_NAME = 'problems_v9.db'
PROBLEM_TABLE = 'problems'
SUBMISSION_TEST_CASE_TABLE = 'submission_test_cases'

class DB:
    def __init__(self, cursor, conn):
        self.cursor = cursor
        self.conn = conn

@contextmanager
def create_sql_connection():
    conn = None
    try:
        conn = sqlite3.connect(DB_NAME)
        conn.row_factory = sqlite3.Row
        yield DB(cursor=conn.cursor(), conn=conn)
    finally:
        if conn:
            conn.close()

def get_problem(problem_id: str):
    with create_sql_connection() as _db:
        return _db.cursor.execute(
            f"SELECT * FROM {PROBLEM_TABLE} WHERE questionId = ?", (problem_id, )
        ).fetchone()  

def get_all_problems():
    with create_sql_connection() as _db:
        return _db.cursor.execute(
            f"SELECT * FROM {PROBLEM_TABLE}"
        ).fetchall()

In [122]:
# dict(get_problem('605'))
[dict(ele) for ele in get_all_problems()]

[{'questionId': '1894',
  'title': 'Merge Strings Alternately',
  'content': 'You are given two strings <code>word1</code> and <code>word2</code>. Merge the strings by adding letters in alternating order, starting with <code>word1</code>. If a string is longer than the other, append the additional letters onto the end of the merged string.\n\n<p>Return <em>the merged string.</em></p>\n\n<p>&nbsp;</p>\n<p><strong class="example">Example 1:</strong></p>\n\n<pre>\n<strong>Input:</strong> word1 = &quot;abc&quot;, word2 = &quot;pqr&quot;\n<strong>Output:</strong> &quot;apbqcr&quot;\n<strong>Explanation:</strong>&nbsp;The merged string will be merged as so:\nword1:  a   b   c\nword2:    p   q   r\nmerged: a p b q c r\n</pre>\n\n<p><strong class="example">Example 2:</strong></p>\n\n<pre>\n<strong>Input:</strong> word1 = &quot;ab&quot;, word2 = &quot;pqrs&quot;\n<strong>Output:</strong> &quot;apbqrs&quot;\n<strong>Explanation:</strong>&nbsp;Notice that as word2 is longer, &quot;rs&quot; is app

In [1]:
import sqlite3
import json

conn = sqlite3.connect('problems_v9.db')
conn.row_factory = sqlite3.Row
cursor = conn.cursor()

In [145]:
SUBMISSION_TEST_CASE_TABLE = "submission_test_cases"
cursor.execute(f'''CREATE TABLE IF NOT EXISTS {SUBMISSION_TEST_CASE_TABLE} (
    questionId TEXT,
    test_case TEXT,
    FOREIGN KEY (questionId) REFERENCES problems(questionId)
)''')
conn.commit()

In [149]:
submission_test_cases = []
for p in cursor.execute('SELECT * FROM problems').fetchall():
#     print(dict(p))
    p = dict(p)
    question_id = p['questionId']
    test_cases = json.loads(p['test_cases'])
    for test_case in test_cases:
        submission_test_cases.append({
            'questionId': question_id,
            'test_case': json.dumps(test_case)
        })

In [150]:
print(len(submission_test_cases))
submission_test_cases[0]

177


{'questionId': '1894',
 'test_case': '{"input_kwargs": {}, "input_args": ["abc", "pqr"], "expected_return": "apbqcr"}'}

In [160]:
updates = [(ele['test_case'], ele['questionId']) for ele in submission_test_cases]
query = f'INSERT INTO {SUBMISSION_TEST_CASE_TABLE} (test_case, questionId) VALUES (?, ?)'
cursor.executemany(query, updates)
conn.commit()

In [161]:
for k in cursor.execute(f'SELECT * FROM {SUBMISSION_TEST_CASE_TABLE}').fetchall():
    k = dict(k)

In [163]:
len(cursor.execute(f'SELECT * FROM {SUBMISSION_TEST_CASE_TABLE}').fetchall())

177

In [250]:
import requests

url = "http://127.0.0.1:8000/submit_code"
data = {
    "problem_id": "1894",
    "code": """class Solution:
    def mergeAlternately(self, word1, word2):
        while True:
            pass
        merged = ""
        min_len = min(len(word1), len(word2))
        for i in range(min_len):
            merged += word1[i] + word2[i]
        merged += word1[min_len:] + word2[min_len:]
        return merged

"""
}

response = requests.post(url, json=data)

print(response.json())


{'output': {'passed': 0, 'total': 3}, 'error': 'Time Limit Exceeded'}


In [183]:
import docker
client = docker.from_env()
client.containers.run('alpine', 'echo hello world')

b'hello world\n'

In [200]:
container = client.containers.run('test_box',
                                      detach=True)

In [186]:
container.remove(force=True)

In [203]:

container = client.containers.run('python:3.9-slim', detach=True, command='tail -f /dev/null')

In [204]:
container.logs()

b''

In [205]:
container.remove(force=True)

In [262]:
from pydantic import BaseModel, field_validator, ValidationError

class User(BaseModel):
    name: str
        
    @field_validator('name')
    @classmethod
    def name_val_func(cls, val):
        if not val:
            raise ValueError('name is invalid')
 
try:
    user = User(name='hi')
except ValidationError as e:
    print(e)

In [270]:
import ast

code = """class Solution:
    def mergeAlternately(self, word1, word2):
        pass"""
tree = ast.parse(code)
print(ast.dump(tree, indent=4))

Module(
    body=[
        ClassDef(
            name='Solution',
            bases=[],
            keywords=[],
            body=[
                FunctionDef(
                    name='mergeAlternately',
                    args=arguments(
                        posonlyargs=[],
                        args=[
                            arg(arg='self'),
                            arg(arg='word1'),
                            arg(arg='word2')],
                        kwonlyargs=[],
                        kw_defaults=[],
                        defaults=[]),
                    body=[
                        Pass()],
                    decorator_list=[])],
            decorator_list=[])],
    type_ignores=[])


In [273]:
import ast

# Original code
code = """
def add(x, y):
    return x + y

result = add(2, 3)
print(result)
"""

# Parse the code into an AST
tree = ast.parse(code)

# Define a transformer to modify the AST
class AddOneTransformer(ast.NodeTransformer):
    def visit_Return(self, node):
        if isinstance(node.value, ast.BinOp) and isinstance(node.value.op, ast.Add):
            # Create a new AST node for `+ 1`
            add_one = ast.BinOp(left=node.value, op=ast.Add(), right=ast.Constant(value=1))
            return ast.Return(value=add_one)
        return node

# Apply the transformation
tree = AddOneTransformer().visit(tree)

# Convert the modified AST back to source code
updated_code = ast.unparse(tree)
print(updated_code)


def add(x, y):
    return x + y + 1
result = add(2, 3)
print(result)


In [302]:
class ProblemUpdate(BaseModel):
    initial_code: str
    
    @field_validator('initial_code')
    @classmethod
    def validate_initial_code(cls, v):
        try:
            tree = ast.parse(v)
            classes = [node for node in ast.walk(tree) if isinstance(node, ast.ClassDef)]
            if not any(cls.name == 'Solution' for cls in classes):
                raise ValueError('initial code must contain Solution class')
            solution_class = [cls for cls in classes if cls.name == 'Solution'][0]
            solution_methods = [node.name for node in solution_class.body if isinstance(node, ast.FunctionDef)]
            print(solution_methods)
            
        except SyntaxError:
            raise ValueError('invalid python syntax in initial code')
        return v
        

In [306]:
initial_code = """class Solution:
    def __call__(self,hey):
        pass
    def mergeAlternately(self, word1, word2):
        pass"""
try:
    problem = ProblemUpdate(initial_code = initial_code)
except Exception as e:
    print(e)

['__call__', 'mergeAlternately']


In [301]:
dir(ast)

['AST',
 'Add',
 'And',
 'AnnAssign',
 'Assert',
 'Assign',
 'AsyncFor',
 'AsyncFunctionDef',
 'AsyncWith',
 'Attribute',
 'AugAssign',
 'AugLoad',
 'AugStore',
 'Await',
 'BinOp',
 'BitAnd',
 'BitOr',
 'BitXor',
 'BoolOp',
 'Break',
 'Bytes',
 'Call',
 'ClassDef',
 'Compare',
 'Constant',
 'Continue',
 'Del',
 'Delete',
 'Dict',
 'DictComp',
 'Div',
 'Ellipsis',
 'Eq',
 'ExceptHandler',
 'Expr',
 'Expression',
 'ExtSlice',
 'FloorDiv',
 'For',
 'FormattedValue',
 'FunctionDef',
 'FunctionType',
 'GeneratorExp',
 'Global',
 'Gt',
 'GtE',
 'If',
 'IfExp',
 'Import',
 'ImportFrom',
 'In',
 'Index',
 'IntEnum',
 'Interactive',
 'Invert',
 'Is',
 'IsNot',
 'JoinedStr',
 'LShift',
 'Lambda',
 'List',
 'ListComp',
 'Load',
 'Lt',
 'LtE',
 'MatMult',
 'Mod',
 'Module',
 'Mult',
 'Name',
 'NameConstant',
 'NamedExpr',
 'NodeTransformer',
 'NodeVisitor',
 'Nonlocal',
 'Not',
 'NotEq',
 'NotIn',
 'Num',
 'Or',
 'Param',
 'Pass',
 'Pow',
 'PyCF_ALLOW_TOP_LEVEL_AWAIT',
 'PyCF_ONLY_AST',
 'PyCF_TYP

In [ ]:
code = """
class Solution:
    def method1(self):
        return [1, 2, 3]
"""

tree = ast.parse(code)

# Find the Solution class
solution_class = next(node for node in ast.walk(tree) if isinstance(node, ast.ClassDef) and node.name == "Solution")

# Find the methods in the Solution class
methods = [node for node in solution_class.body if isinstance(node, ast.FunctionDef)]

# Inspect the return statement in the first method
return_stmt = next(node for node in ast.walk(methods[0]) if isinstance(node, ast.Return))

# Access the elements of the returned list
if isinstance(return_stmt.value, ast.List):
    elements = return_stmt.value.elts
    for element in elements:
        if isinstance(element, ast.Constant):
            print(element.value)  # Output the constant values in the list


In [28]:
from sqlalchemy.ext.asyncio import create_async_engine, AsyncSession
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base

DATABASE_URL = "sqlite+aiosqlite:///./test.db"
engine = create_async_engine(DATABASE_URL, echo=True)
AsyncSessionLocal = sessionmaker(engine, class_=AsyncSession, expire_on_commit=False)
Base = declarative_base()

/var/folders/8k/07c2mpg56x7c31_26k0qj2y00000gn/T/ipykernel_77888/21257270.py:8: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [29]:
from rq import Worker, Queue, Connection


In [31]:
import redis
from rq import Worker, Queue, Connection

listen = ['default']

redis_conn = redis.Redis(host='localhost', port=6379, db=0)

if __name__ == '__main__':
    with Connection(redis_conn):
        worker = Worker(list(map(Queue, listen)))
        worker.work()

09:24:45 Worker rq:worker:168f059d7c554926b322a7239c9dd5d9 started with PID 77888, version 1.16.2
09:24:45 Subscribing to channel rq:pubsub:168f059d7c554926b322a7239c9dd5d9
09:24:45 *** Listening on default...
09:24:45 Cleaning registries for queue: default
09:33:49 default: test2.simple_task(2, 3) (8aa08da6-a309-4488-aa3a-78eea326ff4a)
starting task: add 2 + 3
task completed 2 + 3 = 5
09:33:59 default: Job OK (8aa08da6-a309-4488-aa3a-78eea326ff4a)
09:33:59 Result is kept for 500 seconds
09:34:38 default: test2.simple_task(2, 3) (504b6f24-0765-4e62-b0df-5e09f2326d64)
starting task: add 2 + 3
task completed 2 + 3 = 5
09:34:48 default: Job OK (504b6f24-0765-4e62-b0df-5e09f2326d64)
09:34:48 Result is kept for 500 seconds
09:34:48 Cleaning registries for queue: default
09:34:48 default: test2.simple_task(2, 3) (cabb7773-ad41-4bb6-b28b-99c03381fca5)
starting task: add 2 + 3
task completed 2 + 3 = 5
09:34:58 default: Job OK (cabb7773-ad41-4bb6-b28b-99c03381fca5)
09:34:58 Result is kept for 5

Exception in thread 10:43:10 Could not connect to Redis instance: Connection closed by server. Retrying in 1 seconds...
Thread-5:
Traceback (most recent call last):
  File "/Applications/anaconda3/envs/m/lib/python3.9/threading.py", line 980, in _bootstrap_inner
    self.run()
  File "/Applications/anaconda3/envs/m/lib/python3.9/site-packages/redis/client.py", line 1164, in run
    pubsub.get_message(ignore_subscribe_messages=True, timeout=sleep_time)
  File "/Applications/anaconda3/envs/m/lib/python3.9/site-packages/redis/client.py", line 1026, in get_message
    response = self.parse_response(block=(timeout is None), timeout=timeout)
  File "/Applications/anaconda3/envs/m/lib/python3.9/site-packages/redis/client.py", line 837, in parse_response
    response = self._execute(conn, try_read)
  File "/Applications/anaconda3/envs/m/lib/python3.9/site-packages/redis/client.py", line 813, in _execute
    return conn.retry.call_with_retry(
  File "/Applications/anaconda3/envs/m/lib/python3.9

10:56:10 Worker 168f059d7c554926b322a7239c9dd5d9 [PID 77888]: warm shut down requested


ConnectionError: Error 61 connecting to localhost:6379. Connection refused.

In [29]:
import sqlite3
import json

conn = sqlite3.connect('problems_v9.db')
conn.row_factory = sqlite3.Row
cursor = conn.cursor()

In [13]:
dict(cursor.execute('SELECT * FROM problems WHERE questionId=?', ('1146',)).fetchone())

{'questionId': '1146',
 'title': 'Greatest Common Divisor of Strings',
 'content': '<p>For two strings <code>s</code> and <code>t</code>, we say &quot;<code>t</code> divides <code>s</code>&quot; if and only if <code>s = t + t + t + ... + t + t</code> (i.e., <code>t</code> is concatenated with itself one or more times).</p>\n\n<p>Given two strings <code>str1</code> and <code>str2</code>, return <em>the largest string </em><code>x</code><em> such that </em><code>x</code><em> divides both </em><code>str1</code><em> and </em><code>str2</code>.</p>\n\n<p>&nbsp;</p>\n<p><strong class="example">Example 1:</strong></p>\n\n<pre>\n<strong>Input:</strong> str1 = &quot;ABCABC&quot;, str2 = &quot;ABC&quot;\n<strong>Output:</strong> &quot;ABC&quot;\n</pre>\n\n<p><strong class="example">Example 2:</strong></p>\n\n<pre>\n<strong>Input:</strong> str1 = &quot;ABABAB&quot;, str2 = &quot;ABAB&quot;\n<strong>Output:</strong> &quot;AB&quot;\n</pre>\n\n<p><strong class="example">Example 3:</strong></p>\n\n<p

In [14]:
tamil = """<p>இரண்டு சரங்கள் <code>s</code> மற்றும் <code>t</code> க்கு, "<code>t</code>, <code>s</code> ஐ பிரிக்கிறது" என்று சொல்கிறோம், அப்படி என்றால் <code>s = t + t + t + ... + t + t</code> (அதாவது, <code>t</code> ஐ தன்னுடன் ஒன்று அல்லது அதற்கு மேற்பட்ட முறை இணைக்கப்பட்டுள்ளது).</p>

<p>இரண்டு சரங்கள் <code>str1</code> மற்றும் <code>str2</code> கொடுக்கப்பட்டுள்ளன. <em>மிகப்பெரிய சரம் </em><code>x</code><em> ஐ திருப்பி அனுப்புங்கள், அது </em><code>str1</code><em> மற்றும் </em><code>str2</code><em> இரண்டையும் பிரிக்கிறது</em>.</p> <p>&nbsp;</p> <p><strong class="example">உதாரணம் 1:</strong></p> <pre> <strong>உள்ளீடு:</strong> str1 = "ABCABC", str2 = "ABC" <strong>வெளியீடு:</strong> "ABC" </pre> <p><strong class="example">உதாரணம் 2:</strong></p> <pre> <strong>உள்ளீடு:</strong> str1 = "ABABAB", str2 = "ABAB" <strong>வெளியீடு:</strong> "AB" </pre> <p><strong class="example">உதாரணம் 3:</strong></p> <pre> <strong>உள்ளீடு:</strong> str1 = "LEET", str2 = "CODE" <strong>வெளியீடு:</strong> "" </pre> <p>&nbsp;</p> <p><strong>கட்டுப்பாடுகள்:</strong></p> <ul> <li><code>1 <= str1.length, str2.length <= 1000</code></li> <li><code>str1</code> மற்றும் <code>str2</code> ஆங்கில பெரிய எழுத்துக்களால் மட்டுமே ஆனவை.</li> </ul>"""


In [15]:
hindi = """<p>दो स्ट्रिंग्स <code>s</code> और <code>t</code> के लिए, हम कहते हैं "<code>t</code>, <code>s</code> को विभाजित करता है" यदि और केवल यदि <code>s = t + t + t + ... + t + t</code> (यानी, <code>t</code> को खुद से एक या अधिक बार जोड़ा गया है)।</p>
<p>दो स्ट्रिंग्स <code>str1</code> और <code>str2</code> दी गई हैं। <em>सबसे बड़ी स्ट्रिंग </em><code>x</code><em> लौटाएं जो </em><code>str1</code><em> और </em><code>str2</code><em> दोनों को विभाजित करती है</em>।</p>
<p>&nbsp;</p>
<p><strong class="example">उदाहरण 1:</strong></p>
<pre>
<strong>इनपुट:</strong> str1 = "ABCABC", str2 = "ABC"
<strong>आउटपुट:</strong> "ABC"
</pre>
<p><strong class="example">उदाहरण 2:</strong></p>
<pre>
<strong>इनपुट:</strong> str1 = "ABABAB", str2 = "ABAB"
<strong>आउटपुट:</strong> "AB"
</pre>
<p><strong class="example">उदाहरण 3:</strong></p>
<pre>
<strong>इनपुट:</strong> str1 = "LEET", str2 = "CODE"
<strong>आउटपुट:</strong> ""
</pre>
<p>&nbsp;</p>
<p><strong>प्रतिबंध:</strong></p>
<ul>
	<li><code>1 <= str1.length, str2.length <= 1000</code></li>
	<li><code>str1</code> और <code>str2</code> में केवल अंग्रेजी के बड़े अक्षर होंगे।</li>
</ul>"""

In [16]:
# cursor.execute("ALTER TABLE problems ADD COLUMN content_ta TEXT")
# conn.commit()

# cursor.execute("ALTER TABLE problems ADD COLUMN content_hi TEXT")
# conn.commit()

In [17]:
update_id = '1146'
cursor.execute("UPDATE problems SET content_ta = ? WHERE questionId = ?", (tamil, update_id))
conn.commit()

cursor.execute("UPDATE problems SET content_hi = ? WHERE questionId = ?", (hindi, update_id))
conn.commit()

In [25]:
data_list = [
        {
            "id": 1,
            "banner_img": "assets/banner_basic_algos.jpeg",
            "banner_title": "Basic Algos",
            "banner_description": "Beginner Friendly Basic Algos",
            "isLocked": False,
        },
        {
            "id": 2,
            "banner_img": "assets/banner_basic_ml.jpeg",
            "banner_title": "Basic ML",
            "banner_description": "Beginner Friendly Machine Learning Algos",
            "isLocked": True,
        },
        {
            "id": 3,
            "banner_img": "assets/banner_basic_dl.jpeg",
            "banner_title": "Basic DL",
            "banner_description": "Beginner Friendly Deep Learning Techniques",
            "isLocked": True,
        },
        {
            "id": 4,
            "banner_img":  "assets/banner_power_algo.jpeg",
            "banner_title": "Power Algos",
            "banner_description": "Master Essential Algo Problems",
            "isLocked": True,
        },
        {
            "id": 5,
            "banner_img": "assets/banner_power_ml.jpeg",
            "banner_title": "Power ML",
            "banner_description": "Master Essential Machine Learning Algos",
            "isLocked": True,
        },
        {
            "id": 6,
            "banner_img": "assets/banner_power_dl.jpeg",
            "banner_title": "Power DL",
            "banner_description": "Master Essential Deep Learning Techniques",
            "isLocked": True,
        },
    ]

In [19]:
MODULES_TABLE = "modules"

In [20]:
cursor.execute(f'''
CREATE TABLE {MODULES_TABLE} (
    id INTEGER PRIMARY KEY,
    banner_img TEXT,
    banner_title TEXT,
    banner_description TEXT,
    isLocked BOOLEAN
)
''')
conn.commit()

In [26]:
data_list

[{'id': 1,
  'banner_img': 'assets/banner_basic_algos.jpeg',
  'banner_title': 'Basic Algos',
  'banner_description': 'Beginner Friendly Basic Algos',
  'isLocked': False},
 {'id': 2,
  'banner_img': 'assets/banner_basic_ml.jpeg',
  'banner_title': 'Basic ML',
  'banner_description': 'Beginner Friendly Machine Learning Algos',
  'isLocked': True},
 {'id': 3,
  'banner_img': 'assets/banner_basic_dl.jpeg',
  'banner_title': 'Basic DL',
  'banner_description': 'Beginner Friendly Deep Learning Techniques',
  'isLocked': True},
 {'id': 4,
  'banner_img': 'assets/banner_power_algo.jpeg',
  'banner_title': 'Power Algos',
  'banner_description': 'Master Essential Algo Problems',
  'isLocked': True},
 {'id': 5,
  'banner_img': 'assets/banner_power_ml.jpeg',
  'banner_title': 'Power ML',
  'banner_description': 'Master Essential Machine Learning Algos',
  'isLocked': True},
 {'id': 6,
  'banner_img': 'assets/banner_power_dl.jpeg',
  'banner_title': 'Power DL',
  'banner_description': 'Master Ess

In [30]:
# Insert the data
cursor.executemany(f'''
INSERT INTO {MODULES_TABLE} (id, banner_img, banner_title, banner_description, isLocked)
VALUES (:id, :banner_img, :banner_title, :banner_description, :isLocked)
''', data_list)

# Commit the changes and close the connection
conn.commit()

IntegrityError: UNIQUE constraint failed: modules.id